In [1]:
import os
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt, iirnotch

# Configuration
BASE_PATH = '/kaggle/input/mtcaic3-phase-ii'
OUTPUT_DIR = './preprocessed'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Filter design
def design_filters(fs=250.0):
    # Bandpass 1-40 Hz
    bp_b, bp_a = butter(4, [1/(fs/2), 40/(fs/2)], btype='band')
    # Notch at 50 Hz
    notch_b, notch_a = iirnotch(50/(fs/2), Q=30)
    return bp_b, bp_a, notch_b, notch_a

# Preprocessing steps for one trial
def preprocess_trial(df):
    eeg_cols = ['FZ','C3','CZ','C4','PZ','PO7','OZ','PO8']
    motion_cols = ['AccX','AccY','AccZ','Gyro1','Gyro2','Gyro3']
    val_col = 'Validation'

    # 1) Motion artifact detection
    motion_mag = np.sqrt((df[motion_cols]**2).sum(axis=1))
    mot_thresh = np.percentile(motion_mag, 95)
    bad_mask = motion_mag > mot_thresh

    # 2) Mask EEG
    data = df[eeg_cols].copy().values
    data[bad_mask, :] = np.nan
    data[df[val_col] == 0, :] = np.nan

    # 3) Interpolation
    for ch in range(data.shape[1]):
        col = data[:, ch]
        nans = np.isnan(col)
        if nans.all():
            continue
        idx = np.arange(len(col))
        data[nans, ch] = np.interp(idx[nans], idx[~nans], col[~nans])

    # 4) Filtering
    bp_b, bp_a, notch_b, notch_a = design_filters()
    for ch in range(data.shape[1]):
        data[:, ch] = filtfilt(bp_b, bp_a, data[:, ch])
        data[:, ch] = filtfilt(notch_b, notch_a, data[:, ch])

    # 5) Baseline correction (first 0.5s)
    bs = int(0.5 * 250)
    baseline = data[:bs].mean(axis=0)
    data -= baseline
    return data

# Load index DataFrame
def load_index(fname, label_col=True):
    df = pd.read_csv(os.path.join(BASE_PATH, fname))
    cols = ['id','subject_id','task','trial_session','trial'] + (['label'] if label_col else [])
    return df[cols]

# Process a split, grouping by task
def process_split(df, has_label=True):
    data_dict = {'MI': {'X': [], 'y': []}, 'SSVEP': {'X': [], 'y': []}}
    for _, row in df.iterrows():
        # Identify folder
        idx = row['id']
        split = 'train' if idx <= 4800 else 'validation' if idx <= 4900 else 'test'
        # Load EEGdata
        path = os.path.join(BASE_PATH, row['task'], split, row['subject_id'], str(row['trial_session']), 'EEGdata.csv')
        df_eeg = pd.read_csv(path)
        # Extract correct segment
        n_samp = 2250 if row['task']=='MI' else 1750
        start = (row['trial']-1)*n_samp
        seg = df_eeg.iloc[int(start):int(start+n_samp)].reset_index(drop=True)
        proc = preprocess_trial(seg)
        data_dict[row['task']]['X'].append(proc.T)
        if has_label:
            data_dict[row['task']]['y'].append(row['label'])
    # Stack
    for t in data_dict:
        X = np.stack(data_dict[t]['X'])
        y = np.array(data_dict[t]['y']) if has_label else None
        data_dict[t]['X'] = X
        data_dict[t]['y'] = y
    return data_dict

# Execute processing and save
for fname, label_col in [('train.csv', True), ('validation.csv', True), ('test.csv', False)]:
    df_idx = load_index(fname, label_col)
    results = process_split(df_idx, label_col)
    for task, d in results.items():
        out_file = f"{os.path.splitext(fname)[0]}_{task}.npz"
        path = os.path.join(OUTPUT_DIR, out_file)
        if label_col:
            np.savez_compressed(path, X=d['X'], y=d['y'])
        else:
            np.savez_compressed(path, X=d['X'])
        print(f"Saved {out_file}: X={d['X'].shape}" + (f", y={d['y'].shape}" if d['y'] is not None else ''))
print('Preprocessing complete.')

Saved train_MI.npz: X=(2400, 8, 2250), y=(2400,)
Saved train_SSVEP.npz: X=(2400, 8, 1750), y=(2400,)
Saved validation_MI.npz: X=(50, 8, 2250), y=(50,)
Saved validation_SSVEP.npz: X=(50, 8, 1750), y=(50,)
Saved test_MI.npz: X=(100, 8, 2250)
Saved test_SSVEP.npz: X=(100, 8, 1750)
Preprocessing complete.


In [2]:
import os
import numpy as np
import tensorflow as tf

from sklearn.metrics import f1_score, classification_report
from mne.decoding import CSP
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, LearningRateScheduler

# --- 0) Config ---
data_dir   = './preprocessed'
output_dir = './models3'
os.makedirs(output_dir, exist_ok=True)

eeg_indices = [1, 2, 3, 4]  # C3, CZ, C4, PZ

# --- 1) Load & filter data ---
train_npz = np.load(os.path.join(data_dir, 'train_MI.npz'))
val_npz   = np.load(os.path.join(data_dir, 'validation_MI.npz'))

X_train_all, y_train = train_npz['X'], train_npz['y']
X_val_all,   y_val   = val_npz['X'],   val_npz['y']

# Filter for EEG channels only
X_train_raw = X_train_all[:, eeg_indices, :].transpose(0, 2, 1).astype('float32')  # (n, T, 4)
X_val_raw   = X_val_all[:,   eeg_indices, :].transpose(0, 2, 1).astype('float32')  # (n, T, 4)

# --- 2) Binarize labels ---
y_train_bin = (y_train == 'Right').astype(int)
y_val_bin   = (y_val   == 'Right').astype(int)

# --- 3) CSP for models 3, 4, 5 ---
csp = CSP(n_components=4, log=False, norm_trace=False)
X_train_csp_input = X_train_raw.transpose(0, 2, 1).astype('float64').copy()  # (n, channels, time)
csp.fit(X_train_csp_input, y_train_bin)  # ← NO 'rank' argument

W = csp.filters_[:4]

def apply_csp(X):  # X: (n, T, C)
    return np.stack([W.dot(ep.T) for ep in X], axis=0)


Xtr_csp = apply_csp(X_train_raw).astype('float32')
Xvl_csp = apply_csp(X_val_raw).astype('float32')

Xtr_csp = Xtr_csp.transpose(0, 2, 1)  # (n, T, 4)
Xvl_csp = Xvl_csp.transpose(0, 2, 1)


# For 2D models
Xtr_csp_2d = Xtr_csp[..., np.newaxis]
Xvl_csp_2d = Xvl_csp[..., np.newaxis]

# --- 4) One-hot labels ---
ytr_oh = keras.utils.to_categorical(y_train_bin, 2)
yvl_oh = keras.utils.to_categorical(y_val_bin,   2)

# --- 5) Data augmentation ---
def aug_gen(X, y, seed=0, batch_size=32):
    n = X.shape[0]
    rng = np.random.RandomState(seed)
    while True:
        idx = rng.randint(0, n, batch_size)
        bx, by = X[idx].copy(), y[idx]
        bx += rng.normal(0, 0.005, bx.shape)
        yield bx, by

train_gen_raw   = aug_gen(X_train_raw,  ytr_oh, seed=0, batch_size=64)
train_gen_csp1d = aug_gen(Xtr_csp,      ytr_oh, seed=1, batch_size=64)
train_gen_csp2d = aug_gen(Xtr_csp_2d,   ytr_oh, seed=2, batch_size=64)

steps_raw   = len(X_train_raw)  // 64
steps_csp1d = len(Xtr_csp)      // 64
steps_csp2d = len(Xtr_csp_2d)   // 64

# --- 6) Cosine LR schedule ---
def cosine_lr(epoch, lr_max=5e-5, epochs=200):
    return lr_max * (1 + np.cos(np.pi * epoch / epochs)) / 2

# --- 7) F1 Score Metric ---
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name="f1_score", **kwargs):
        super().__init__(name=name, **kwargs)
        self.tp = self.add_weight(name="tp", initializer="zeros")
        self.fp = self.add_weight(name="fp", initializer="zeros")
        self.fn = self.add_weight(name="fn", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        preds  = tf.argmax(y_pred, axis=1)
        labels = tf.argmax(y_true, axis=1)
        self.tp.assign_add(tf.reduce_sum(tf.cast(tf.logical_and(preds == 1, labels == 1), tf.float32)))
        self.fp.assign_add(tf.reduce_sum(tf.cast(tf.logical_and(preds == 1, labels == 0), tf.float32)))
        self.fn.assign_add(tf.reduce_sum(tf.cast(tf.logical_and(preds == 0, labels == 1), tf.float32)))

    def result(self):
        p = self.tp / (self.tp + self.fp + tf.keras.backend.epsilon())
        r = self.tp / (self.tp + self.fn + tf.keras.backend.epsilon())
        return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

    def reset_states(self):
        self.tp.assign(0.0)
        self.fp.assign(0.0)
        self.fn.assign(0.0)

# --- 8) Callback factory ---
def get_callbacks(name):
    return [
        EarlyStopping("val_f1_score", mode="max", patience=20, restore_best_weights=True),
        ModelCheckpoint(os.path.join(output_dir, f"best_{name}.h5"),
                        "val_f1_score", mode="max", save_best_only=True),
        CSVLogger(os.path.join(output_dir, f"log_{name}.csv")),
        LearningRateScheduler(cosine_lr)
    ]

# --- 9) Model Builders ---
# [All model builder functions remain unchanged — see previous message if needed.]
def build_modelA(input_shape):
    m = keras.Sequential([
        layers.Input(input_shape),
        layers.Conv1D(32, 5, activation="relu", padding="same"),
        layers.BatchNormalization(), layers.MaxPool1D(2),
        layers.Conv1D(64, 5, activation="relu", padding="same"),
        layers.BatchNormalization(), layers.MaxPool1D(2),
        layers.Conv1D(128,5,activation="relu",padding="same"),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling1D(),
        layers.Dense(64, activation="relu", 
                     kernel_regularizer=regularizers.l2(1e-4)),
        layers.Dropout(0.7),
        layers.Dense(2, activation="softmax"),
    ])
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_modelB(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for f in [16,32,64,128,256]:
        x = layers.Conv1D(f,3,activation="relu",padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPool1D(2)(x)
    x = layers.Flatten()(x)
    for u in [128,64,32]:
        x = layers.Dense(u, activation="relu",
                         kernel_regularizer=regularizers.l2(1e-4))(x)
        x = layers.Dropout(0.5)(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp, out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model1(input_shape):
    inp = layers.Input(input_shape+(1,))
    x = layers.Concatenate()([inp, inp, inp])
    x = layers.Resizing(32,32)(x)
    base = keras.applications.ResNet50(
        include_top=False, weights="imagenet",
        input_shape=(32,32,3), pooling="avg"
    )
    base.trainable = False
    x = base(x)
    x = layers.Reshape((1, x.shape[-1]))(x)
    for _ in range(7):
        x = layers.Conv1D(64,3,activation="relu",padding="same")(x)
    x = layers.MultiHeadAttention(num_heads=4,key_dim=32)(x,x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64,activation="relu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model2(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for _ in range(3):
        x = layers.Conv1D(32,3,activation="elu",padding="same")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64,activation="elu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model3(input_shape):
    inp = layers.Input(input_shape)  # (T, F, 1)
    x = inp
    for _ in range(5):
        x = layers.Conv2D(32,(3,3),activation="relu",padding="same")(x)
    x = layers.Flatten()(x)
    for u in [128,64,32]:
        x = layers.Dense(u, activation="relu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model4(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for _ in range(3):
        x = layers.Conv1D(64,3,activation="relu",padding="same")(x)
    x = layers.LSTM(128)(x)
    for _ in range(4):
        x = layers.Dense(64, activation="relu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model5(input_shape):
    inp = layers.Input(input_shape)
    x = inp
    for _ in range(7):
        x = layers.Conv1D(64,3,activation="elu",padding="same")(x)
    x = layers.Flatten()(x)
    for _ in range(3):
        x = layers.Dense(64, activation="elu")(x)
    out = layers.Dense(2, activation="softmax")(x)
    m = models.Model(inp,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m

def build_model6(input_shape):
    C3, C4 = 0,2
    eeg_in = layers.Input(input_shape)
    c3 = layers.Lambda(lambda x: x[:,:,C3:C3+1])(eeg_in)
    c4 = layers.Lambda(lambda x: x[:,:,C4:C4+1])(eeg_in)
    def branch():
        return models.Sequential([
            layers.Conv1D(16,250,activation="relu",padding="same"),
            layers.MaxPool1D(3),
            layers.Conv1D(32,50,activation="relu",padding="same"),
            layers.GlobalAveragePooling1D()
        ])
    b3, b4 = branch()(c3), branch()(c4)
    x = layers.Concatenate()([b3,b4])
    for _ in range(4):
        x = layers.Dense(64,activation="relu")(x)
    out = layers.Dense(2,activation="softmax")(x)
    m = models.Model(eeg_in,out)
    m.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy", F1Score()])
    return m
# (You can paste model builders here unchanged if needed)

# --- 10) Train & evaluate ---
builders = {
    'oldA': build_modelA,
    'oldB': build_modelB,
    'model1': build_model1,
    'model2': build_model2,
    'model3': build_model3,
    'model4': build_model4,
    'model5': build_model5,
    'model6': build_model6,
}


2025-07-23 04:59:37.534621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753246777.720990      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753246777.773235      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Computing rank from data with rank=None
    Using tolerance 3.2e+03 (2.2e-16 eps * 4 dim * 3.6e+18  max singular value)
    Estimated rank (data): 4
    data: rank 4 computed from 4 data channels with 0 projectors
Reducing data rank from 4 -> 4
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.


In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import f1_score, classification_report
from mne.decoding import CSP
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, LearningRateScheduler
import gc

# --- 0) Config ---
BASE_PATH = '/kaggle/input/mtcaic3-phase-ii'  # Update this path
data_dir = './preprocessed'
output_dir = './models_loso'
os.makedirs(output_dir, exist_ok=True)

eeg_indices = [1, 2, 3, 4]  # C3, CZ, C4, PZ
batch_size = 64

# --- 1) Load & prepare data ---
# Load subject information from train.csv
train_df = pd.read_csv(os.path.join(BASE_PATH, 'train.csv'))
mi_train_df = train_df[train_df['task'] == 'MI'].reset_index(drop=True)
subject_ids = mi_train_df['subject_id'].values

# Load preprocessed data
train_npz = np.load(os.path.join(data_dir, 'train_MI.npz'))
X_train_all = train_npz['X']  # (2400, 8, 2250)
y_train = train_npz['y']      # (2400,)

# Filter for EEG channels
X_train_raw = X_train_all[:, eeg_indices, :].transpose(0, 2, 1).astype('float32')  # (2400, 2250, 4)

# Binarize labels
y_train_bin = (y_train == 'Right').astype(int)

# --- 2) Setup LOSO Cross-Validation ---
logo = LeaveOneGroupOut()
groups = subject_ids
n_folds = logo.get_n_splits(groups=groups)

# --- 3) Model Builders (Unchanged from original) ---
# [Include all model builder functions: build_modelA, build_modelB, build_model1, etc.]
# ... (Paste all your model builder functions here unchanged) ...

# --- 4) F1 Score Metric (Unchanged) ---
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name="f1_score", **kwargs):
        super().__init__(name=name, **kwargs)
        self.tp = self.add_weight(name="tp", initializer="zeros")
        self.fp = self.add_weight(name="fp", initializer="zeros")
        self.fn = self.add_weight(name="fn", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        preds  = tf.argmax(y_pred, axis=1)
        labels = tf.argmax(y_true, axis=1)
        self.tp.assign_add(tf.reduce_sum(tf.cast(tf.logical_and(preds == 1, labels == 1), tf.float32)))
        self.fp.assign_add(tf.reduce_sum(tf.cast(tf.logical_and(preds == 1, labels == 0), tf.float32)))
        self.fn.assign_add(tf.reduce_sum(tf.cast(tf.logical_and(preds == 0, labels == 1), tf.float32)))

    def result(self):
        p = self.tp / (self.tp + self.fp + tf.keras.backend.epsilon())
        r = self.tp / (self.tp + self.fn + tf.keras.backend.epsilon())
        return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

    def reset_states(self):
        self.tp.assign(0.0)
        self.fp.assign(0.0)
        self.fn.assign(0.0)

    # ... (Paste your F1Score class implementation here) ...

# --- 5) Cosine LR Schedule (Unchanged) ---
def cosine_lr(epoch, lr_max=5e-5, epochs=200):
    return lr_max * (1 + np.cos(np.pi * epoch / epochs)) / 2

# --- 6) Callback Factory ---
def get_callbacks(name, fold_dir):
    return [
        EarlyStopping(monitor="val_f1_score", mode="max", patience=20, restore_best_weights=True),
        ModelCheckpoint(os.path.join(fold_dir, f"best_{name}.h5")),
        CSVLogger(os.path.join(fold_dir, f"log_{name}.csv")),
        LearningRateScheduler(cosine_lr)
    ]

# --- 7) Data Augmentation Generator ---
def aug_gen(X, y, seed=0, batch_size=32):
    n = X.shape[0]
    rng = np.random.RandomState(seed)
    while True:
        idx = rng.randint(0, n, batch_size)
        bx, by = X[idx].copy(), y[idx]
        bx += rng.normal(0, 0.005, bx.shape)
        yield bx, by

# --- 8) LOSO Cross-Validation ---
# Dictionary to store results
builders = {
    'oldA': build_modelA,
    'oldB': build_modelB,
    'model1': build_model1,
    'model2': build_model2,
    'model3': build_model3,
    'model4': build_model4,
    'model5': build_model5,
    'model6': build_model6,
}

# Initialize results storage
results = {name: [] for name in builders.keys()}

for fold_idx, (train_idx, val_idx) in enumerate(logo.split(X_train_raw, y_train_bin, groups=groups)):
    subject_id = groups[val_idx[0]]
    print(f"\n{'='*50}")
    print(f"Fold {fold_idx+1}/{n_folds} - Subject: {subject_id}")
    print(f"{'='*50}")
    
    # Create output directory for fold
    fold_dir = os.path.join(output_dir, f"fold_{fold_idx+1}")
    os.makedirs(fold_dir, exist_ok=True)
    
    # Split data
    X_train_fold_raw = X_train_raw[train_idx]
    y_train_fold_bin = y_train_bin[train_idx]
    X_val_fold_raw = X_train_raw[val_idx]
    y_val_fold_bin = y_train_bin[val_idx]
    
    # --- Compute CSP using ONLY training fold data ---
    print("Computing CSP for current fold...")
    csp = CSP(n_components=4, log=False, norm_trace=False)
    
    # Prepare data for CSP: (n_trials, n_channels, n_times)
    X_train_csp_input = X_train_fold_raw.transpose(0, 2, 1).astype('float64')
    csp.fit(X_train_csp_input, y_train_fold_bin)
    W = csp.filters_[:4]  # (4, 4) filter matrix
    
    # Apply CSP to both training and validation data
    def apply_csp_fold(X):
        return np.stack([W.T @ x.T for x in X], axis=0)  # (n, n_components, n_times)
    
    X_train_fold_csp = apply_csp_fold(X_train_fold_raw).transpose(0, 2, 1).astype('float32')
    X_val_fold_csp = apply_csp_fold(X_val_fold_raw).transpose(0, 2, 1).astype('float32')
    
    # Create 2D versions for models that need it
    X_train_fold_csp_2d = X_train_fold_csp[..., np.newaxis]
    X_val_fold_csp_2d = X_val_fold_csp[..., np.newaxis]
    
    # One-hot encode labels
    y_train_fold_oh = keras.utils.to_categorical(y_train_fold_bin, 2)
    y_val_fold_oh = keras.utils.to_categorical(y_val_fold_bin, 2)
    
    # Steps per epoch
    steps_per_epoch = len(X_train_fold_raw) // batch_size
    
    # --- Train all models on current fold ---
    for name, build_fn in builders.items():
        print(f"\n>>> Training {name} on Fold {fold_idx+1}")
        
        # Clean up previous models
        tf.keras.backend.clear_session()
        gc.collect()
        
        # Select appropriate data format
        if name == 'model3':
            X_train = X_train_fold_csp_2d
            X_val = X_val_fold_csp_2d
            input_shape = X_train.shape[1:]
            train_gen = aug_gen(X_train, y_train_fold_oh, seed=fold_idx, batch_size=batch_size)
        elif name in ['model4', 'model5']:
            X_train = X_train_fold_csp
            X_val = X_val_fold_csp
            input_shape = X_train.shape[1:]
            train_gen = aug_gen(X_train, y_train_fold_oh, seed=fold_idx, batch_size=batch_size)
        else:
            X_train = X_train_fold_raw
            X_val = X_val_fold_raw
            input_shape = X_train.shape[1:]
            train_gen = aug_gen(X_train, y_train_fold_oh, seed=fold_idx, batch_size=batch_size)
        
        # Build model
        model = build_fn(input_shape)
        
        # Train model
        history = model.fit(
            train_gen,
            steps_per_epoch=steps_per_epoch,
            epochs=200,
            validation_data=(X_val, y_val_fold_oh),
            callbacks=get_callbacks(name, fold_dir),
            verbose=2
        )
        
        # Evaluate
        preds = np.argmax(model.predict(X_val), axis=1)
        f1 = f1_score(y_val_fold_bin, preds)
        print(f"{name} → Fold {fold_idx+1} Val F1 = {f1:.4f}")
        results[name].append(f1)
        
        # Save fold results
        np.save(os.path.join(fold_dir, f"results_{name}.npy"), np.array([f1]))

# --- 9) Final Results ---
print("\n\n=== Final LOSO Results ===")
for name, f1_scores in results.items():
    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)
    print(f"{name}:")
    print(f"  F1 Scores: {', '.join([f'{s:.4f}' for s in f1_scores])}")
    print(f"  Mean F1 = {mean_f1:.4f} ± {std_f1:.4f}\n")

# Save all results
np.save(os.path.join(output_dir, "all_results.npy"), results)
print("LOSO complete!")


Fold 1/30 - Subject: S1
Computing CSP for current fold...
Computing rank from data with rank=None
    Using tolerance 3.2e+03 (2.2e-16 eps * 4 dim * 3.6e+18  max singular value)
    Estimated rank (data): 4
    data: rank 4 computed from 4 data channels with 0 projectors
Reducing data rank from 4 -> 4
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.

>>> Training oldA on Fold 1


I0000 00:00:1753246800.576299      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Epoch 1/200


I0000 00:00:1753246805.840761      61 service.cc:148] XLA service 0x7913a8024d10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753246805.841518      61 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753246806.292436      61 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753246809.929072      61 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


36/36 - 10s - 289ms/step - accuracy: 0.4991 - f1_score: 0.5301 - loss: 0.8895 - val_accuracy: 0.4375 - val_f1_score: 0.4000 - val_loss: 0.7517 - learning_rate: 0.0010
Epoch 2/200
36/36 - 1s - 28ms/step - accuracy: 0.5234 - f1_score: 0.5059 - loss: 0.7833 - val_accuracy: 0.4625 - val_f1_score: 0.4267 - val_loss: 0.7272 - learning_rate: 9.9994e-04
Epoch 3/200
36/36 - 1s - 28ms/step - accuracy: 0.5087 - f1_score: 0.5116 - loss: 0.7438 - val_accuracy: 0.5625 - val_f1_score: 0.2553 - val_loss: 0.6991 - learning_rate: 9.9969e-04
Epoch 4/200
36/36 - 1s - 28ms/step - accuracy: 0.5200 - f1_score: 0.5102 - loss: 0.7230 - val_accuracy: 0.5500 - val_f1_score: 0.4000 - val_loss: 0.6985 - learning_rate: 9.9914e-04
Epoch 5/200
36/36 - 1s - 28ms/step - accuracy: 0.5078 - f1_score: 0.5057 - loss: 0.7210 - val_accuracy: 0.5250 - val_f1_score: 0.3448 - val_loss: 0.7048 - learning_rate: 9.9815e-04
Epoch 6/200
36/36 - 1s - 28ms/step - accuracy: 0.4965 - f1_score: 0.5089 - loss: 0.7141 - val_accuracy: 0.550

ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 211, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python3.11/dist-pac